# CFP Propensity Version 1.0

### LIBRARY IMPORTS

In [1]:
'''Importing Required Libraries'''
import copy
from collections import OrderedDict
import pandas as pd
import lightgbm as lgb
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.options.display.float_format = '{:.2f}'.format
pd.options.mode.chained_assignment = None  # default='warn'
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pandas.plotting import scatter_matrix
from matplotlib import pyplot
from scipy import stats
from scipy.stats import norm, skew
from sklearn.preprocessing import PowerTransformer, QuantileTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.linear_model import LogisticRegression, SGDClassifier,SGDRegressor
from sklearn.ensemble import RandomForestClassifier,VotingClassifier
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import accuracy_score, roc_curve, roc_auc_score, confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.pipeline import make_pipeline
import joblib

### DATA IMPORT

In [2]:
'''Importing the data for modelling'''
# cfp_data = pd.read_csv(r"D:\Dbeaver Out\CFP_TrainData_Green_DecJan_202302131645.csv", na_values=['',' ','NH', 'NA', '[]'], low_memory=False)
cfp_data = pd.read_csv(r"D:\Dbeaver Out\CFP_Data_RedLtd2302_3Fixed_202302231212.csv", na_values=['',' ','NH', 'NA', '[]'], low_memory=False)

In [3]:
cfp_data.drop(['reff_88','reff_07', 'ref_390', 'ref_prod_disb'],axis=1,inplace=True) 

In [4]:
cfp_data.columns

Index(['user_id', 'customer_type', 'nsaleable', 'latest_login_date',
       'salary_account', 'monthly_income', 'credit_score', 'age', 'pincode',
       'marital_status', 'employment_type', 'first_profile_date',
       'latest_profile_date', 'city_band', 'ptp_last_three_months_flag',
       'positive_response_rate', 'contactability', 'email_login_flag',
       'unique_days_logged_in_flag', 'sms_login_flag', 'last_login_vintage',
       'negative_status_flag', 'settled_flag', 'open_total_ratio',
       'totl_neg_ratio', 'saleable_accounts_count', 'cfp_interest',
       'auto_loan_count', 'gold_loan_count', 'consumer_loan_count',
       'housing_loan_count', 'credit_card_count', 'personal_loan_count',
       'two_wheeler_count', 'open_accounts', 'open_bank_fintech',
       'open_bank_nbfc', 'closed_accounts', 'settled_accounts',
       'written_off_accounts', 'flows_accounts', 'secured', 'unsecured',
       'open_negative_status_count', 'individual_account', 'joint_account',
       'tota

In [5]:
'''Capping'''
cfp_data['age'] = np.where(cfp_data['age']<16,16,cfp_data['age'])
cfp_data['age'] = np.where(cfp_data['age']>42,42,cfp_data['age'])
cfp_data['monthly_income'] = np.where(cfp_data['monthly_income']>33000,33000,cfp_data['monthly_income'])
cfp_data['credit_score'] = np.where(cfp_data['credit_score']>750,750,cfp_data['credit_score'])
cfp_data['auto_loan_count'] = np.where(cfp_data['auto_loan_count']>2,2,cfp_data['auto_loan_count'])
cfp_data['gold_loan_count'] = np.where(cfp_data['gold_loan_count']>9,9,cfp_data['gold_loan_count'])
# cfp_data['consumer_loan_count'] = np.where(cfp_data['consumer_loan_count']>3,3,cfp_data['consumer_loan_count'])
cfp_data['housing_loan_count'] = np.where(cfp_data['housing_loan_count']>1,1,cfp_data['housing_loan_count'])
# cfp_data['credit_card_count'] = np.where(cfp_data['credit_card_count']>4,4,cfp_data['credit_card_count'])
# cfp_data['personal_loan_count'] = np.where(cfp_data['personal_loan_count']>2,2,cfp_data['personal_loan_count'])
cfp_data['two_wheeler_count'] = np.where(cfp_data['two_wheeler_count']>2,2,cfp_data['two_wheeler_count'])
# cfp_data['open_accounts'] = np.where(cfp_data['open_accounts']>7,7,cfp_data['open_accounts'])
cfp_data['open_bank_fintech'] = np.where(cfp_data['open_bank_fintech']>1,1,cfp_data['open_bank_fintech'])
cfp_data['open_bank_nbfc'] = np.where(cfp_data['open_bank_nbfc']>2,2,cfp_data['open_bank_nbfc'])
# cfp_data['closed_accounts'] = np.where(cfp_data['closed_accounts']>5,5,cfp_data['closed_accounts'])
cfp_data['settled_accounts'] = np.where(cfp_data['settled_accounts']>5,5,cfp_data['settled_accounts'])
cfp_data['written_off_accounts'] = np.where(cfp_data['written_off_accounts']>5,5,cfp_data['written_off_accounts'])
cfp_data['flows_accounts'] = np.where(cfp_data['flows_accounts']>5,5,cfp_data['flows_accounts'])
# cfp_data['secured'] = np.where(cfp_data['secured']>1,1,cfp_data['secured'])
# cfp_data['unsecured'] = np.where(cfp_data['unsecured']>1,1,cfp_data['unsecured'])
cfp_data['open_negative_status_count'] = np.where(cfp_data['open_negative_status_count']>5,5,cfp_data['open_negative_status_count'])
cfp_data['individual_account'] = np.where(cfp_data['individual_account']>5,5,cfp_data['individual_account'])
cfp_data['joint_account'] = np.where(cfp_data['joint_account']>5,5,cfp_data['joint_account'])
cfp_data['total_amt_sanctioned'] = np.where(cfp_data['total_amt_sanctioned']>200,200,cfp_data['total_amt_sanctioned'])
cfp_data['total_balance'] = np.where(cfp_data['total_balance']>50,50,cfp_data['total_balance'])
cfp_data['credit_limit'] = np.where(cfp_data['credit_limit']>20,20,cfp_data['credit_limit'])
cfp_data['repayment_tenure'] = np.where(cfp_data['repayment_tenure']>12,12,cfp_data['repayment_tenure'])
cfp_data['total_emi_amount'] = np.where(cfp_data['total_emi_amount']>20000,20000,cfp_data['total_emi_amount'])
cfp_data['hdfc_neg_flag'] = np.where(cfp_data['hdfc_neg_flag']>1,1,cfp_data['hdfc_neg_flag'])
cfp_data['citi_neg_flag'] = np.where(cfp_data['citi_neg_flag']>1,1,cfp_data['citi_neg_flag'])
cfp_data['sbi_neg_flag'] = np.where(cfp_data['sbi_neg_flag']>1,1,cfp_data['sbi_neg_flag'])
cfp_data['icici_neg_flag'] = np.where(cfp_data['icici_neg_flag']>1,1,cfp_data['icici_neg_flag'])
cfp_data['cc_neg_flag'] = np.where(cfp_data['cc_neg_flag']>1,1,cfp_data['cc_neg_flag'])
cfp_data['pl_neg_flag'] = np.where(cfp_data['pl_neg_flag']>1,1,cfp_data['pl_neg_flag'])
cfp_data['cl_neg_flag'] = np.where(cfp_data['cl_neg_flag']>1,1,cfp_data['cl_neg_flag'])
cfp_data['auto_loan_close_count_last3mnths'] = np.where(cfp_data['auto_loan_close_count_last3mnths']>1,1,cfp_data['auto_loan_close_count_last3mnths'])
cfp_data['gold_loan_close_count_last3mnths'] = np.where(cfp_data['gold_loan_close_count_last3mnths']>1,1,cfp_data['gold_loan_close_count_last3mnths'])
cfp_data['consumer_loan_close_count_last3mnths'] = np.where(cfp_data['consumer_loan_close_count_last3mnths']>1,1,cfp_data['consumer_loan_close_count_last3mnths'])
cfp_data['housing_loan_close_count_last3mnths'] = np.where(cfp_data['housing_loan_close_count_last3mnths']>1,1,cfp_data['housing_loan_close_count_last3mnths'])
cfp_data['credit_card_close_count_last3mnths'] = np.where(cfp_data['credit_card_close_count_last3mnths']>1,1,cfp_data['credit_card_close_count_last3mnths'])
cfp_data['personal_loan_close_count_last3mnths'] = np.where(cfp_data['personal_loan_close_count_last3mnths']>1,1,cfp_data['personal_loan_close_count_last3mnths'])
cfp_data['two_wheeler_close_count_last3mnths'] = np.where(cfp_data['two_wheeler_close_count_last3mnths']>1,1,cfp_data['two_wheeler_close_count_last3mnths'])
cfp_data['individual_close_account_last3mnths'] = np.where(cfp_data['individual_close_account_last3mnths']>1,1,cfp_data['individual_close_account_last3mnths'])
cfp_data['closed_accounts_last3mnths'] = np.where(cfp_data['closed_accounts_last3mnths']>1,1,cfp_data['closed_accounts_last3mnths'])
cfp_data['secured_close_last3mnths'] = np.where(cfp_data['secured_close_last3mnths']>1,1,cfp_data['secured_close_last3mnths'])
cfp_data['unsecured_close_last3mnths'] = np.where(cfp_data['unsecured_close_last3mnths']>1,1,cfp_data['unsecured_close_last3mnths'])
cfp_data['negative_status_close_count_last3mnths'] = np.where(cfp_data['negative_status_close_count_last3mnths']>1,1,cfp_data['negative_status_close_count_last3mnths'])
cfp_data['joint_account_close_last3mnths'] = np.where(cfp_data['joint_account_close_last3mnths']>1,1,cfp_data['joint_account_close_last3mnths'])
cfp_data['max_score'] = np.where(cfp_data['max_score']>750,750,cfp_data['max_score'])
cfp_data['score_on_m9'] = np.where(cfp_data['score_on_m9']>750,750,cfp_data['score_on_m9'])
cfp_data['score_on_m3'] = np.where(cfp_data['score_on_m3']>750,750,cfp_data['score_on_m3'])
cfp_data['score_on_m12'] = np.where(cfp_data['score_on_m12']>750,750,cfp_data['score_on_m12'])
cfp_data['score_on_m6'] = np.where(cfp_data['score_on_m6']>750,750,cfp_data['score_on_m6'])
cfp_data['neg_acc_on_m9'] = np.where(cfp_data['neg_acc_on_m9']>6,6,cfp_data['neg_acc_on_m9'])
cfp_data['neg_acc_on_m3'] = np.where(cfp_data['neg_acc_on_m3']>6,6,cfp_data['neg_acc_on_m3'])
cfp_data['neg_acc_on_m12'] = np.where(cfp_data['neg_acc_on_m12']>6,6,cfp_data['neg_acc_on_m12'])
cfp_data['neg_acc_on_m6'] = np.where(cfp_data['neg_acc_on_m6']>6,6,cfp_data['neg_acc_on_m6'])
# cfp_data['reff_88'] = np.where(cfp_data['reff_88']>2,2,cfp_data['reff_88'])
# cfp_data['reff_07'] = np.where(cfp_data['reff_07']>2,2,cfp_data['reff_07'])
# cfp_data['ref_390'] = np.where(cfp_data['ref_390']>1,1,cfp_data['ref_390'])
# cfp_data['ref_prod_disb'] = np.where(cfp_data['ref_prod_disb']>4,4,cfp_data['ref_prod_disb'])

## PRE-PROCESSING

#### Handling Null Values

In [6]:
cfp_data.isnull().sum()

user_id                                         0
customer_type                                   0
nsaleable                                       0
latest_login_date                           44278
salary_account                            1527281
monthly_income                               8082
credit_score                                    0
age                                            74
pincode                                       149
marital_status                             134917
employment_type                            402377
first_profile_date                              0
latest_profile_date                             0
city_band                                       0
ptp_last_three_months_flag                1581938
positive_response_rate                          0
contactability                                  0
email_login_flag                                0
unique_days_logged_in_flag                      0
sms_login_flag                                  0


In [7]:
''' Replacing with 0s and Harcoded Values'''
cfp_data.ptp_last_three_months_flag.fillna(0, inplace= True)
cfp_data.totl_neg_ratio.fillna(0, inplace= True)
cfp_data.max_score.fillna(cfp_data.max_score.mean(),inplace=True)
cfp_data.auto_loan_close_count_last3mnths.fillna(0, inplace= True)
cfp_data.gold_loan_close_count_last3mnths.fillna(0, inplace= True)
cfp_data.consumer_loan_close_count_last3mnths.fillna(0, inplace= True)
cfp_data.housing_loan_close_count_last3mnths.fillna(0, inplace= True)
cfp_data.credit_card_close_count_last3mnths.fillna(0, inplace= True)
cfp_data.personal_loan_close_count_last3mnths.fillna(0, inplace= True)
cfp_data.two_wheeler_close_count_last3mnths.fillna(0, inplace= True)
cfp_data.individual_close_account_last3mnths.fillna(0, inplace= True)
cfp_data.secured_close_last3mnths.fillna(0, inplace= True)
cfp_data.closed_accounts_last3mnths.fillna(0, inplace= True)
cfp_data.negative_status_close_count_last3mnths.fillna(0, inplace= True)
cfp_data.joint_account_close_last3mnths.fillna(0, inplace= True)
cfp_data.unsecured_close_last3mnths.fillna(0, inplace= True)

cfp_data.auto_loan_count.fillna(0, inplace= True)
cfp_data.gold_loan_count.fillna(0, inplace= True)

cfp_data.housing_loan_count.fillna(0, inplace= True)
cfp_data.credit_card_count.fillna(0, inplace= True)
cfp_data.personal_loan_count.fillna(0, inplace= True)
cfp_data.two_wheeler_count.fillna(0, inplace= True)
cfp_data.individual_account.fillna(0, inplace= True)
cfp_data.open_accounts.fillna(0, inplace= True)
cfp_data.settled_accounts.fillna(0, inplace= True)
cfp_data.secured.fillna(0, inplace= True)
cfp_data.unsecured.fillna(0, inplace= True)
cfp_data.open_negative_status_count.fillna(0, inplace= True)
cfp_data.joint_account.fillna(0, inplace= True)
cfp_data.total_balance.fillna(0, inplace= True)
cfp_data.written_off_accounts.fillna(0, inplace= True)
cfp_data.flows_accounts.fillna(0, inplace= True)
cfp_data.open_accounts.fillna(0, inplace= True)
cfp_data.closed_accounts.fillna(0, inplace= True)


cfp_data.hdfc_neg_flag.fillna(0, inplace= True)
cfp_data.citi_neg_flag.fillna(0, inplace= True)
cfp_data.sbi_neg_flag.fillna(0, inplace= True)        
cfp_data.icici_neg_flag.fillna(0, inplace= True)         
cfp_data.cc_neg_flag.fillna(0, inplace= True)       
cfp_data.pl_neg_flag.fillna(0, inplace= True)
cfp_data.cl_neg_flag.fillna(0, inplace= True)

cfp_data.total_balance.fillna(0,inplace=True)
cfp_data.total_emi_amount.fillna(0,inplace=True)
cfp_data.total_amt_sanctioned.fillna(0,inplace=True)

cfp_data.credit_limit.fillna(0, inplace= True)
cfp_data.repayment_tenure.fillna(0, inplace= True)


cfp_data.cfp_interest.fillna(0, inplace= True)

cfp_data.salary_account.fillna('Unknown', inplace=True)

In [8]:
'''Replacing based on median'''
median_marriage_age=36
def marital_na_filler(marital_status, age): 
    try: 
        if np.isnan(marital_status): 
            if age >= median_marriage_age: 
                return 'Married' 
            else: 
                return 'Single' 
        else: 
            return (marital_status) 
    except: 
        return (marital_status)
cfp_data.marital_status = cfp_data.apply(lambda x: marital_na_filler(x.marital_status, x.age), axis = 1)
cfp_data.monthly_income.fillna(40000, inplace= True)
cfp_data.score_on_m9.fillna(650, inplace= True)
cfp_data.score_on_m3.fillna(650, inplace= True)
cfp_data.score_on_m12.fillna(650, inplace= True)
cfp_data.score_on_m6.fillna(650, inplace= True)
cfp_data.neg_acc_on_m9.fillna(1, inplace= True)
cfp_data.neg_acc_on_m3.fillna(1, inplace= True)
cfp_data.neg_acc_on_m12.fillna(1, inplace= True)
cfp_data.neg_acc_on_m6.fillna(1, inplace= True)
cfp_data.max_dpd_on_m9.fillna(0, inplace= True)
cfp_data.max_dpd_on_m3.fillna(0, inplace= True)
cfp_data.max_dpd_on_m12.fillna(0, inplace= True)
cfp_data.max_dpd_on_m6.fillna(0, inplace= True)
cfp_data.age.fillna(32, inplace= True)

In [9]:
''' Replacing FLAG in city_band with 'Missing/Others' '''
cfp_data.city_band = cfp_data.city_band.apply(lambda x: 'Missing/Others' if x == 'FLAG' else x)

In [10]:
'''Combining Score Params'''
def score_track(credit_score,score_on_m3,score_on_m6,score_on_m9):
    try:
        if (credit_score < score_on_m3):
            return 'Last 3 mths declining'
        if (credit_score <= score_on_m3) & (score_on_m3 > score_on_m6) & (score_on_m6 <= score_on_m9):
            return 'Last 6 mths declining'
        if (credit_score <= score_on_m3) & (score_on_m3 <= score_on_m6):
            return 'Last 6 mths declining'
        if (credit_score <= score_on_m3) & (score_on_m3 <= score_on_m6) & (score_on_m6 <= score_on_m9):
            return 'Last 9 mths declining'      
        if (credit_score > score_on_m3): 
            return 'Last 3 mths Score improving'
    except:
        return 'Score Inconsistent'
    
cfp_data['score_track'] = cfp_data.apply(lambda x: score_track(x['credit_score'] ,x['score_on_m3'] ,x['score_on_m6'],x['score_on_m9']) , axis = 1)

In [11]:
'''Combining Neg Acc Params'''
def neg_acc_track(neg_acc_on_m3,neg_acc_on_m6,neg_acc_on_m9,neg_acc_on_m12):
    try:
        if (neg_acc_on_m3 >= neg_acc_on_m6):
            return 'Last 6 mths increasing'
        if (neg_acc_on_m3 >= neg_acc_on_m6) & (neg_acc_on_m6 >= neg_acc_on_m9):
            return 'Last 9 mths increasing'
        if (neg_acc_on_m3 >= neg_acc_on_m6) & (neg_acc_on_m6 >= neg_acc_on_m9) & (neg_acc_on_m6 >= neg_acc_on_m12):
            return 'Last 12 mths increasing'
    except:
        return 'Neg acc Inconsistent'
    
cfp_data['neg_acc_track'] = cfp_data.apply(lambda x: neg_acc_track(x['neg_acc_on_m3'] ,x['neg_acc_on_m6'] ,x['neg_acc_on_m9'],x['neg_acc_on_m12']) , axis = 1)

In [12]:
'''Combining DPD Params'''
def max_dpd_track(max_dpd_on_m3,max_dpd_on_m6,max_dpd_on_m9,max_dpd_on_m12):
    try:
        if (max_dpd_on_m3 >= max_dpd_on_m6):
            return 'Last 6 mths increasing'
        if (max_dpd_on_m3 >= max_dpd_on_m6) & (max_dpd_on_m6 >= max_dpd_on_m9):
            return 'Last 9 mths increasing'
        if (max_dpd_on_m3 >= max_dpd_on_m6) & (max_dpd_on_m6 >= max_dpd_on_m9) & (max_dpd_on_m6 >= max_dpd_on_m9):
            return 'Last 12 mths increasing'
    except:
        return 'Neg acc Inconsistent'
    
cfp_data['max_dpd_track'] = cfp_data.apply(lambda x: max_dpd_track(x['max_dpd_on_m3'] ,x['max_dpd_on_m6'] ,x['max_dpd_on_m9'],x['max_dpd_on_m12']) , axis = 1)

In [13]:
''' Open_accounts_flag '''
def open_accounts_flag(open_accounts):
    try:
        if open_accounts == 0:
            return 'Zero'
        elif open_accounts == 1:
            return 'One'
        elif open_accounts == 2:
            return 'Two'
        elif open_accounts == 3:
            return 'Three'
        elif open_accounts == 4:
            return 'Four'
        elif open_accounts == 5:
            return 'Five'
        else:
            return 'MoreThanFive'
    except:
        return 'Flag'

cfp_data['open_accounts_cat'] = cfp_data['open_accounts'].apply(lambda x: open_accounts_flag(x))

In [14]:
''' close_accounts_flag '''
def closed_accounts_flag(closed_accounts):
    try:
        if closed_accounts == 0:
            return 'Zero'
        elif closed_accounts == 1:
            return 'One'
        elif closed_accounts == 2:
            return 'Two'
        elif closed_accounts == 3:
            return 'Three'
        elif closed_accounts == 4:
            return 'Four'
        elif closed_accounts == 5:
            return 'Five'
        else:
            return 'MoreThanFive'
    except:
        return 'Flag'
cfp_data['closed_accounts_cat'] = cfp_data['closed_accounts'].apply(lambda x: closed_accounts_flag(x))

In [15]:
''' credit_card_flag '''
def credit_card_flag(credit_card_count):
    try:
        if credit_card_count == 0:
            return 'Zero'
        elif credit_card_count == 1:
            return 'One'
        elif credit_card_count == 2:
            return 'Two'
        elif credit_card_count == 3:
            return 'Three'
        elif credit_card_count == 4:
            return 'Four'
        elif credit_card_count == 5:
            return 'Five'
        else:
            return 'MoreThanFive'
    except:
        return 'Flag'
cfp_data['credit_card_cat'] = cfp_data['credit_card_count'].apply(lambda x: credit_card_flag(x))    

In [16]:
''' personal_loan_flag '''
def personal_loan_flag(personal_loan_count):
    try:
        if personal_loan_count == 0:
            return 'Zero'
        elif personal_loan_count == 1:
            return 'One'
        else:
            return 'MoreThanOne'
    except:
        return 'Flag'
cfp_data['personal_loan_cat'] = cfp_data['personal_loan_count'].apply(lambda x: personal_loan_flag(x))   

In [17]:
''' consumer_loan_flag '''
def consumer_loan_flag(consumer_loan_count):
    try:
        if consumer_loan_count == 0:
            return 'Zero'
        elif consumer_loan_count == 1:
            return 'One'
        elif consumer_loan_count == 2:
            return 'Two'
        else:
            return 'MoreThanTwo'
    except:
        return 'Flag'
cfp_data['consumer_loan_cat'] = cfp_data['consumer_loan_count'].apply(lambda x: consumer_loan_flag(x))   

In [18]:
''' secured_accounts_flag '''
def secured_accounts_flag(secured):
    try:
        if secured == 0:
            return 'Zero'
        elif secured == 1:
            return 'One'
        elif secured == 2:
            return 'Two'
        elif secured == 3:
            return 'Three'
        elif secured == 4:
            return 'Four'
        elif secured == 5:
            return 'Five'
        else:
            return 'MoreThanFive'
    except:
        return 'Flag'
cfp_data['secured_accounts_cat'] = cfp_data['secured'].apply(lambda x: secured_accounts_flag(x))

In [19]:
''' unsecured_accounts_flag '''
def unsecured_accounts_flag(unsecured):
    try:
        if unsecured == 0:
            return 'Zero'
        elif unsecured == 1:
            return 'One'
        elif unsecured == 2:
            return 'Two'
        elif unsecured == 3:
            return 'Three'
        elif unsecured == 4:
            return 'Four'
        elif unsecured == 5:
            return 'Five'
        elif unsecured == 6:
            return 'Six'
        elif unsecured == 7:
            return 'Seven'
        elif unsecured == 8:
            return 'Eight'
        elif unsecured == 9:
            return 'Nine'
        elif unsecured == 10:
            return 'Ten'
        else:
            return 'MoreThanTen'
    except:
        return 'Flag'
cfp_data['unsecured_accounts_cat'] = cfp_data['unsecured'].apply(lambda x: unsecured_accounts_flag(x))

In [20]:
# ''' Dropping the columns that we changed above '''
# drops = ['score_on_m9',
#        'score_on_m3', 'score_on_m12', 'score_on_m6',
#        'neg_acc_on_m9', 'neg_acc_on_m3', 'neg_acc_on_m12',
#         'max_dpd_on_m9', 'max_dpd_on_m3',
#        'max_dpd_on_m12', 'max_dpd_on_m6',  
#        'neg_acc_on_m6','secured','unsecured','open_accounts','closed_accounts',
#         'credit_card_count','personal_loan_count','consumer_loan_count']
# cfp_data.drop(drops, axis = 1, inplace = True)

In [21]:
'''Login Vintage'''
def getFormattedLoginVintage(Login):
    try:
        if Login == 'M1toM2':
            return 3
        elif Login == 'M3toM6':
            return 2
        elif Login == 'M7toM12':
            return 1
        elif Login == 'BeyondM12':
            return 0
    except:
        return 0
cfp_data.last_login_vintage = cfp_data.last_login_vintage.apply(lambda x: getFormattedLoginVintage(x))
cfp_data.last_login_vintage.value_counts()

0    900921
2    284622
1    268536
3    253066
Name: last_login_vintage, dtype: int64

In [22]:
'''Replacing Salary Account Bank with Categories'''
def getFormattedSalaryAccount(Bank):
    try:
        formatted_employment_type = ""
        SBI = ['SBI GROUP','SBI']
        AXIS = ['AXIS','Axis']
        HDFC = ['HDFC','HDFC Bank']
        KOTAK = ['kotak mahindra bank','KOTAK','kotak']
        BOI = ['BANK OF INDIA']
        PNB = ['PUNJAB NATIONAL', 'PNB']
        if Bank in SBI:
            return "Public"
        elif Bank in AXIS:
            return "Private"
        elif Bank in HDFC:
            return "Private"
        elif Bank in KOTAK:
            return "Private"
        elif Bank in PNB:
            return "Private"
        elif Bank in BOI:
            return "Public"
        elif Bank == "YESBANK":
            return "Private"
        elif Bank == "ANDHRA":
            return "Public"
        elif Bank == "IDBI":
            return "Private"
        elif Bank == "CANARA":
            return "Public"
        elif Bank == "UNION":
            return "Public"
        elif Bank == "ICICI":
            return "Private"
        elif Bank == "receiveByCashOrCheque":
            return "I receive by cash"
        elif Bank == "CITI":
            return "Private"
        elif Bank == 'I receive by cash':
            return "I receive by cash"
        elif Bank == 'I receive by cheque':
            return "I receive by cash"
        elif Bank == 'Unknown':
            return "Unknown"
        else:
            return 'Other'
    except:
        return employment_type
cfp_data.salary_account = cfp_data.salary_account.apply(lambda x: getFormattedSalaryAccount(x))
cfp_data.salary_account.value_counts()

Unknown              1527281
Other                  79515
Private                59754
Public                 39138
I receive by cash       1457
Name: salary_account, dtype: int64

In [23]:
'''Format Employment and handle null values'''
def getFormattedEmploymentType(employment_type):
    try:
        formatted_employment_type = ""
        salaried_list = ['Salaried','salaried','Salaried Doctor','Working Executive','Student','Salaried doctor', 'salaried', 'salariedDoctor']
        self_employed_list = ['Self Employed', 'Self employed', 'Self employed professional', 'Self-Employed', 'selfemployee', 'selfEmployedProfessional','Self employed business', 'selfEmployedBusiness', 'Self Employed Business']
        if employment_type in salaried_list:
            return "Salaried"
        elif employment_type in self_employed_list:
            return "Self Employed"
        else:
            return 'Salaried'
    except:
        return employment_type
cfp_data.employment_type = cfp_data.employment_type.apply(lambda x: getFormattedEmploymentType(x))

In [24]:
''' Income and Age to positive if negative '''
cfp_data.monthly_income = cfp_data.monthly_income.apply(lambda x: (x * -1) if x < 0 else x)
cfp_data.age = cfp_data.age.apply(lambda x: (x * -1) if x < 0 else x)

In [25]:
cfp_data.isnull().sum()

user_id                                        0
customer_type                                  0
nsaleable                                      0
latest_login_date                          44278
salary_account                                 0
monthly_income                                 0
credit_score                                   0
age                                            0
pincode                                      149
marital_status                                 0
employment_type                                0
first_profile_date                             0
latest_profile_date                            0
city_band                                      0
ptp_last_three_months_flag                     0
positive_response_rate                         0
contactability                                 0
email_login_flag                               0
unique_days_logged_in_flag                     0
sms_login_flag                                 0
last_login_vintage  

### Pickling Data

In [26]:
# joblib.dump(cfp_data, 'cfp_clean_data_Green.pkl')
# joblib.dump(cfp_data, 'cfp_clean_data_Red.pkl')
joblib.dump(cfp_data, 'cfp_clean_data_Red_ltd.pkl')
# joblib.dump(cfp_data, 'cfp_clean_data_Green_ltd.pkl')

['cfp_clean_data_Red_ltd.pkl']

In [27]:
# cfp_data=joblib.load('cfp_clean_data_Green.pkl')
# cfp_data=joblib.load('cfp_clean_data_Red.pkl')
cfp_data=joblib.load('cfp_clean_data_Red_ltd.pkl')
# cfp_data=joblib.load('cfp_clean_data_Green_ltd.pkl')

In [28]:
cfp_backup=copy.deepcopy(cfp_data)
cfp_data.drop(['latest_login_date','first_profile_date','latest_profile_date','pincode','customer_type'],axis=1,inplace=True) 

#### Probe and Handle Outliers

In [29]:
exclude=['user_id','cfp_interest','positive_response_rate', 'contactability',
         'ptp_last_three_months_flag','cfp_subscribed', 'ltd', 'nsaleable', 'latest_login_date',
       'salary_account', 'credit_score',
        'pincode', 'marital_status', 'employment_type',
       'first_profile_date', 'latest_profile_date', 'city_band',
       'positive_response_rate', 'email_login_flag', 'unique_days_logged_in_flag',
       'sms_login_flag', 'last_login_vintage', 'negative_status_flag',
       'settled_flag', 'open_total_ratio', 'totl_neg_ratio',
       'saleable_accounts_count', 'cfp_interest', 'auto_loan_count',
       'gold_loan_count', 'housing_loan_count', 'two_wheeler_count',
       'open_bank_fintech', 'open_bank_nbfc', 'settled_accounts',
       'written_off_accounts', 'flows_accounts', 'open_negative_status_count',
       'individual_account', 'joint_account',
       'hdfc_neg_flag', 'citi_neg_flag', 'sbi_neg_flag', 'icici_neg_flag',
       'cc_neg_flag', 'pl_neg_flag', 'cl_neg_flag',
       'auto_loan_close_count_last3mnths', 'gold_loan_close_count_last3mnths',
       'consumer_loan_close_count_last3mnths',
       'housing_loan_close_count_last3mnths',
       'credit_card_close_count_last3mnths',
       'personal_loan_close_count_last3mnths',
       'two_wheeler_close_count_last3mnths',
       'individual_close_account_last3mnths', 'closed_accounts_last3mnths',
       'secured_close_last3mnths', 'unsecured_close_last3mnths',
       'negative_status_close_count_last3mnths',
       'joint_account_close_last3mnths', 'max_score', 'personal_loan_cat',
       'consumer_loan_cat', 'secured_accounts_cat', 'unsecured_accounts_cat'] #,'cfp_subscribed'

In [30]:
numerical_columns=list(cfp_data.select_dtypes(include='number').columns)
for feature in numerical_columns:
    if feature in exclude:
        pass
    else:
        numerical_columns.remove(feature)
print(numerical_columns)

['user_id', 'nsaleable', 'credit_score', 'ptp_last_three_months_flag', 'positive_response_rate', 'contactability', 'last_login_vintage', 'settled_flag', 'open_total_ratio', 'totl_neg_ratio', 'saleable_accounts_count', 'cfp_interest', 'auto_loan_count', 'gold_loan_count', 'housing_loan_count', 'two_wheeler_count', 'open_bank_fintech', 'open_bank_nbfc', 'settled_accounts', 'written_off_accounts', 'flows_accounts', 'open_negative_status_count', 'individual_account', 'joint_account', 'total_balance', 'repayment_tenure', 'hdfc_neg_flag', 'citi_neg_flag', 'sbi_neg_flag', 'icici_neg_flag', 'cc_neg_flag', 'pl_neg_flag', 'cl_neg_flag', 'auto_loan_close_count_last3mnths', 'gold_loan_close_count_last3mnths', 'consumer_loan_close_count_last3mnths', 'housing_loan_close_count_last3mnths', 'credit_card_close_count_last3mnths', 'personal_loan_close_count_last3mnths', 'two_wheeler_close_count_last3mnths', 'individual_close_account_last3mnths', 'closed_accounts_last3mnths', 'secured_close_last3mnths', '

In [31]:
'''Finding Skewness'''
skewed=[]
def find_skew(data,numerical_columns):
    skew_list={}
    for feature in numerical_columns:
        try:
            try:
                skew_value=(3*(data[feature].mean()-data[feature].median()))/data[feature].std()
                if ((round(skew_value,2)<-3) or (round(skew_value,2)>3)):
                    skew_list[feature]='Skewwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwww'
                    skewed.append(feature)
                else:
                    skew_list[feature]=round(skew_value,2)
            except ZeroDivisionError:
                skew_list[feature]='Skewwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwww'
                skewed.append(feature)
        except:
            pass
    return skew_list
skew_list=find_skew(cfp_data,numerical_columns)
skewed

['nsaleable', 'saleable_accounts_count', 'two_wheeler_count']

In [32]:
'''Transforming features - Adjusting for skewness'''
for i in numerical_columns:
    if i in exclude:
        pass
    else:
#         if i in ['monthly_income','total_emi_amount','repayment_tenure','credit_limit']:
        try:
            cfp_data[i] =stats.boxcox(array)
        except:
            pass
find_skew(cfp_data,numerical_columns)

{'user_id': -0.17,
 'nsaleable': 'Skewwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwww',
 'credit_score': -0.94,
 'ptp_last_three_months_flag': 0.16,
 'positive_response_rate': 0.51,
 'contactability': 0.52,
 'last_login_vintage': 2.48,
 'settled_flag': 1.9,
 'open_total_ratio': 1.67,
 'totl_neg_ratio': 1.5,
 'saleable_accounts_count': 'Skewwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwww',
 'cfp_interest': 0.84,
 'auto_loan_count': 0.72,
 'gold_loan_count': 0.82,
 'housing_loan_count': 0.63,
 'two_wheeler_count': 'Skewwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwww',
 'open_bank_fintech': 0.02,
 'open_bank_nbfc': 0.1,
 'settled_accounts': 1.75,
 'written_off_accounts': 1.04,
 'flows_accounts': 1.7,
 'open_negative_status_count': 1.54,
 'individual_account': 1.97,
 'joint_account': 0.71,
 'total_balance': 1.5,
 'repayment_tenure': 1.75,
 'hdfc_neg_flag': 0.37,
 'citi_neg_flag': 0.03,
 'sbi_neg_flag': 0.35,
 'icici_neg_flag': 0.15,
 'cc_neg_flag': 0.54,
 'pl_

In [33]:
cfp_data.head()

,user_id,nsaleable,salary_account,monthly_income,credit_score,age,marital_status,employment_type,city_band,ptp_last_three_months_flag,positive_response_rate,contactability,email_login_flag,unique_days_logged_in_flag,sms_login_flag,last_login_vintage,negative_status_flag,settled_flag,open_total_ratio,totl_neg_ratio,saleable_accounts_count,cfp_interest,auto_loan_count,gold_loan_count,housing_loan_count,two_wheeler_count,open_bank_fintech,open_bank_nbfc,settled_accounts,written_off_accounts,flows_accounts,open_negative_status_count,individual_account,joint_account,total_amt_sanctioned,total_balance,credit_limit,repayment_tenure,total_emi_amount,hdfc_neg_flag,citi_neg_flag,sbi_neg_flag,icici_neg_flag,cc_neg_flag,pl_neg_flag,cl_neg_flag,auto_loan_close_count_last3mnths,gold_loan_close_count_last3mnths,consumer_loan_close_count_last3mnths,housing_loan_close_count_last3mnths,credit_card_close_count_last3mnths,personal_loan_close_count_last3mnths,two_wheeler_close_count_last3mnths,individual_close_account_last3mnths,closed_accounts_last3mnths,secured_close_last3mnths,unsecured_close_last3mnths,negative_status_close_count_last3mnths,joint_account_close_last3mnths,max_score,score_track,neg_acc_track,max_dpd_track,open_accounts_cat,closed_accounts_cat,credit_card_cat,personal_loan_cat,consumer_loan_cat,secured_accounts_cat,unsecured_accounts_cat
0,18009582,0,Unknown,16678.58,725,42.00,Married,Salaried,Missing/Others,0.00,0.00,0.00,Zero,Zero,Zero,3,MoreThanThree,0,0.00,0.00,0,0.00,0.00,0.00,0.00,0.00,0,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,725.00,Last 3 mths Score improving,Last 6 mths increasing,Last 6 mths increasing,Zero,Zero,Zero,Zero,MoreThanTwo,Zero,Zero
1,4161279,0,Unknown,29166.67,525,40.00,Married,Self Employed,Missing/Others,0.00,0.00,0.00,Zero,Zero,Zero,3,MoreThanThree,0,0.00,0.00,0,0.00,0.00,0.00,0.00,0.00,0,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,750.00,Last 3 mths declining,Last 6 mths increasing,Last 6 mths increasing,Zero,Zero,Zero,Zero,MoreThanTwo,Zero,Zero
2,22573946,0,Unknown,17011.60,650,27.00,Single,Self Employed,Delhi/NCR,0.00,1.00,1.00,MoreThanOnce,MoreThanThree,One,3,MoreThanThree,0,0.00,0.00,0,1.00,0.00,0.00,0.00,0.00,0,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,750.00,Last 3 mths declining,Last 6 mths increasing,Last 6 mths increasing,Zero,Zero,Zero,Zero,MoreThanTwo,Zero,Zero
3,28960462,0,Unknown,0.00,500,23.00,Single,Salaried,Missing/Others,0.00,0.00,0.00,Zero,Zero,Zero,3,MoreThanThree,0,0.00,0.00,0,0.00,0.00,0.00,0.00,0.00,0,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,677.04,Last 3 mths declining,Last 6 mths increasing,Last 6 mths increasing,Zero,Zero,Zero,Zero,MoreThanTwo,Zero,Zero
4,21559518,0,Unknown,23259.31,675,42.00,Married,Salaried,Bangalore,0.00,0.00,0.00,MoreThanOnce,MoreThanThree,MoreThanOnce,3,MoreThanThree,0,0.00,0.00,0,0.00,0.00,0.00,0.00,0.00,0,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,675.00,Last 6 mths declining,Last 6 mths increasing,Last 6 mths increasing,Zero,Zero,Zero,Zero,MoreThanTwo,Zero,Zero


In [34]:
'''Removing too skewed'''
cfp_data.drop(['nsaleable','saleable_accounts_count','two_wheeler_count'],axis=1,inplace=True)
find_skew(cfp_data,numerical_columns)

{'user_id': -0.17,
 'credit_score': -0.94,
 'ptp_last_three_months_flag': 0.16,
 'positive_response_rate': 0.51,
 'contactability': 0.52,
 'last_login_vintage': 2.48,
 'settled_flag': 1.9,
 'open_total_ratio': 1.67,
 'totl_neg_ratio': 1.5,
 'cfp_interest': 0.84,
 'auto_loan_count': 0.72,
 'gold_loan_count': 0.82,
 'housing_loan_count': 0.63,
 'open_bank_fintech': 0.02,
 'open_bank_nbfc': 0.1,
 'settled_accounts': 1.75,
 'written_off_accounts': 1.04,
 'flows_accounts': 1.7,
 'open_negative_status_count': 1.54,
 'individual_account': 1.97,
 'joint_account': 0.71,
 'total_balance': 1.5,
 'repayment_tenure': 1.75,
 'hdfc_neg_flag': 0.37,
 'citi_neg_flag': 0.03,
 'sbi_neg_flag': 0.35,
 'icici_neg_flag': 0.15,
 'cc_neg_flag': 0.54,
 'pl_neg_flag': 1.25,
 'cl_neg_flag': 0.39,
 'auto_loan_close_count_last3mnths': 0.04,
 'gold_loan_close_count_last3mnths': 0.14,
 'consumer_loan_close_count_last3mnths': 0.12,
 'housing_loan_close_count_last3mnths': 0.02,
 'credit_card_close_count_last3mnths': 0.

### Pickling Data

In [35]:
# joblib.dump(cfp_data, 'cfp_transformed_data_Green.pkl')
# joblib.dump(cfp_data, 'cfp_transformed_data_Red.pkl')
joblib.dump(cfp_data, 'cfp_transformed_data_Red_ltd.pkl')
# joblib.dump(cfp_data, 'cfp_transformed_data_Green_ltd.pkl')

['cfp_transformed_data_Red_ltd.pkl']

In [36]:
# cfp_clean_data=joblib.load('cfp_transformed_data_Green.pkl')
# cfp_clean_data=joblib.load('cfp_transformed_data_Red.pkl')
cfp_clean_data=joblib.load('cfp_transformed_data_Red_ltd.pkl')
# cfp_clean_data=joblib.load('cfp_transformed_data_Green_ltd.pkl')

In [37]:
cfp_clean_data.head()

,user_id,salary_account,monthly_income,credit_score,age,marital_status,employment_type,city_band,ptp_last_three_months_flag,positive_response_rate,contactability,email_login_flag,unique_days_logged_in_flag,sms_login_flag,last_login_vintage,negative_status_flag,settled_flag,open_total_ratio,totl_neg_ratio,cfp_interest,auto_loan_count,gold_loan_count,housing_loan_count,open_bank_fintech,open_bank_nbfc,settled_accounts,written_off_accounts,flows_accounts,open_negative_status_count,individual_account,joint_account,total_amt_sanctioned,total_balance,credit_limit,repayment_tenure,total_emi_amount,hdfc_neg_flag,citi_neg_flag,sbi_neg_flag,icici_neg_flag,cc_neg_flag,pl_neg_flag,cl_neg_flag,auto_loan_close_count_last3mnths,gold_loan_close_count_last3mnths,consumer_loan_close_count_last3mnths,housing_loan_close_count_last3mnths,credit_card_close_count_last3mnths,personal_loan_close_count_last3mnths,two_wheeler_close_count_last3mnths,individual_close_account_last3mnths,closed_accounts_last3mnths,secured_close_last3mnths,unsecured_close_last3mnths,negative_status_close_count_last3mnths,joint_account_close_last3mnths,max_score,score_track,neg_acc_track,max_dpd_track,open_accounts_cat,closed_accounts_cat,credit_card_cat,personal_loan_cat,consumer_loan_cat,secured_accounts_cat,unsecured_accounts_cat
0,18009582,Unknown,16678.58,725,42.00,Married,Salaried,Missing/Others,0.00,0.00,0.00,Zero,Zero,Zero,3,MoreThanThree,0,0.00,0.00,0.00,0.00,0.00,0.00,0,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,725.00,Last 3 mths Score improving,Last 6 mths increasing,Last 6 mths increasing,Zero,Zero,Zero,Zero,MoreThanTwo,Zero,Zero
1,4161279,Unknown,29166.67,525,40.00,Married,Self Employed,Missing/Others,0.00,0.00,0.00,Zero,Zero,Zero,3,MoreThanThree,0,0.00,0.00,0.00,0.00,0.00,0.00,0,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,750.00,Last 3 mths declining,Last 6 mths increasing,Last 6 mths increasing,Zero,Zero,Zero,Zero,MoreThanTwo,Zero,Zero
2,22573946,Unknown,17011.60,650,27.00,Single,Self Employed,Delhi/NCR,0.00,1.00,1.00,MoreThanOnce,MoreThanThree,One,3,MoreThanThree,0,0.00,0.00,1.00,0.00,0.00,0.00,0,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,750.00,Last 3 mths declining,Last 6 mths increasing,Last 6 mths increasing,Zero,Zero,Zero,Zero,MoreThanTwo,Zero,Zero
3,28960462,Unknown,0.00,500,23.00,Single,Salaried,Missing/Others,0.00,0.00,0.00,Zero,Zero,Zero,3,MoreThanThree,0,0.00,0.00,0.00,0.00,0.00,0.00,0,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,677.04,Last 3 mths declining,Last 6 mths increasing,Last 6 mths increasing,Zero,Zero,Zero,Zero,MoreThanTwo,Zero,Zero
4,21559518,Unknown,23259.31,675,42.00,Married,Salaried,Bangalore,0.00,0.00,0.00,MoreThanOnce,MoreThanThree,MoreThanOnce,3,MoreThanThree,0,0.00,0.00,0.00,0.00,0.00,0.00,0,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,675.00,Last 6 mths declining,Last 6 mths increasing,Last 6 mths increasing,Zero,Zero,Zero,Zero,MoreThanTwo,Zero,Zero


### Removing Unwanted Columns

In [38]:
user_ids=pd.concat([cfp_clean_data['user_id'].reset_index(drop = True)], axis = 1)
cfp_clean_data=cfp_clean_data.drop(['user_id'],axis=1)

### Segregate Columns for Encoding and Scaling

In [39]:
exemption_features=['cfp_subscribed']
id_columns=['user_id']

In [40]:
categorical_features=list(cfp_clean_data.select_dtypes(exclude='number').columns)
categorical_features

['salary_account',
 'marital_status',
 'employment_type',
 'city_band',
 'email_login_flag',
 'unique_days_logged_in_flag',
 'sms_login_flag',
 'negative_status_flag',
 'score_track',
 'neg_acc_track',
 'max_dpd_track',
 'open_accounts_cat',
 'closed_accounts_cat',
 'credit_card_cat',
 'personal_loan_cat',
 'consumer_loan_cat',
 'secured_accounts_cat',
 'unsecured_accounts_cat']

In [41]:
numeric_features=list(cfp_clean_data.select_dtypes(include='number').columns)
numeric_features
flag_features=[]
# for feature in exemption_features:
#     numeric_features.remove(feature)
for feature in exclude:
    if feature in numeric_features:
        flag_features.append(feature)
        numeric_features.remove(feature)
    

In [42]:
'''Encoding categorical features with dummies'''
dummies = pd.get_dummies(data = cfp_clean_data[categorical_features], drop_first=False)
dummies.drop(['unique_days_logged_in_flag_Zero'],axis=1,inplace=True)
# dummies.drop(['salary_account_CANARA'],axis=1,inplace=True) #Only for green ltd
dummies.columns

Index(['salary_account_I receive by cash', 'salary_account_Other',
       'salary_account_Private', 'salary_account_Public',
       'salary_account_Unknown', 'marital_status_Married',
       'marital_status_Single', 'employment_type_Salaried',
       'employment_type_Self Employed', 'city_band_Bangalore',
       'city_band_Chennai', 'city_band_Delhi/NCR', 'city_band_Hyderabad',
       'city_band_Kolkata', 'city_band_Missing/Others', 'city_band_Mumbai',
       'city_band_Pune', 'email_login_flag_MoreThanOnce',
       'email_login_flag_One', 'email_login_flag_Zero',
       'unique_days_logged_in_flag_MoreThanThree',
       'unique_days_logged_in_flag_One', 'unique_days_logged_in_flag_Three',
       'unique_days_logged_in_flag_Two', 'sms_login_flag_MoreThanOnce',
       'sms_login_flag_One', 'sms_login_flag_Zero',
       'negative_status_flag_MoreThanThree', 'negative_status_flag_One',
       'negative_status_flag_Three', 'negative_status_flag_Two',
       'score_track_Last 3 mths Score i

In [43]:
'''Scaling numeric features'''
standard_scalar = StandardScaler(with_mean=True,with_std=True)
standard_scalar.fit(cfp_clean_data[numeric_features])
numeric_data_scaled = standard_scalar.transform(cfp_clean_data[numeric_features])
numeric_data_scaled = pd.DataFrame(numeric_data_scaled,columns=numeric_features)
numeric_data_scaled.head()

,monthly_income,age,total_amt_sanctioned,total_balance,credit_limit,repayment_tenure,total_emi_amount
0,-0.51,1.32,-0.48,-0.50,-0.27,-0.58,-0.51
1,1.03,1.03,-0.48,-0.50,-0.27,-0.58,-0.51
2,-0.47,-0.91,-0.48,-0.50,-0.27,-0.58,-0.51
3,-2.57,-1.50,-0.48,-0.50,-0.27,-0.58,-0.51
4,0.30,1.32,-0.48,-0.50,-0.27,-0.58,-0.51


In [44]:
'''Concatenating all features'''
processed_dataset = pd.concat([user_ids[id_columns].reset_index(drop = True), 
                               numeric_data_scaled.reset_index(drop = True),
                               cfp_clean_data[flag_features].reset_index(drop = True),
                               dummies.reset_index(drop = True),
#                               cfp_clean_data[exemption_features].reset_index(drop = True)
                              ], axis=1)
processed_dataset.shape

(1707145, 132)

In [45]:
'''Replace Infinite'''
processed_dataset = processed_dataset.replace([np.inf, -np.inf], np.nan).dropna(axis=0)
processed_dataset.shape

(1707145, 132)

In [46]:
'''  Preparing test data '''
data=processed_dataset[['User_id','monthly_income','contactability','positive_response_rate','open_total_ratio','totl_neg_ratio','age','total_balance','total_emi_amount','max_score','credit_score','cfp_interest','unique_days_logged_in_flag_MoreThanThree','credit_limit','last_login_vintage','ptp_last_three_months_flag','gold_loan_count','repayment_tenure','sms_login_flag_MoreThanOnce']]
data.drop_duplicates(inplace = True)

KeyError: "['User_id'] not in index"

##### Pickling Model

In [ ]:
grid2=joblib.load('clf_Ensembled_F_Red_LGBM.pkl')
# grid2=joblib.load('clf_Ensembled_F_Green_Stochastic.pkl')


In [ ]:
grid2.best_estimator_

### Predicting

##### Predicting the test data

In [ ]:
''' Predicting using the best fit parameters '''

test_fit = grid2.predict(data.loc[:,data.columns != 'user_id'])
y_pred=pd.DataFrame(test_fit)


### Split to Deciles

In [ ]:
# y_pred_deciles.drop(y_pred_deciles.index, inplace=True)
y_pred_deciles = pd.concat([y_pred[1].reset_index(drop = True), data['user_id'].reset_index(drop = True)], axis = 1)
y_pred_deciles.rename(columns = {1:'predicted_value'}, inplace = True)
y_pred_deciles.head()

In [ ]:
# y_pred_deciles = pd.merge(y_pred_deciles,cfp_backup, on = 'user_id', how = 'left')
# y_pred_deciles.head()

In [ ]:
y_pred_deciles['decile'] = pd.qcut(y_pred_deciles['predicted_value'].rank(method='first'), 10, labels=False)
y_pred_deciles['decile'] = y_pred_deciles['decile'].apply(lambda x: int(10 - x))
y_pred_deciles['decile'].value_counts()

In [ ]:
dummies1 = pd.get_dummies(data = y_pred_deciles['decile'], drop_first=False)
dummies1.head()
processed_dataset1 = pd.concat([data.reset_index(drop = True),
                               dummies1.reset_index(drop = True)], axis=1)
processed_dataset1.corr(method='pearson').style.format("{:.2}").background_gradient(cmap=plt.get_cmap('coolwarm'), axis=1)

#### Export to CSV

In [ ]:
y_pred_deciles.to_csv('CFP_deciled_data_Red_ltd_Randomized.csv')
# y_pred_deciles.to_csv('CFP_deciled_data_Green_ltd_Randomized.csv')

##### -- The End